<a href="https://colab.research.google.com/github/RafsanJany-44/ARC_Machine2/blob/main/Tinnitus_PreTraining_Tunning_Hybride.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Function

In [42]:
!pip install imbalanced-learn
!pip install ipywidgets

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from imblearn.over_sampling import SMOTE
import ipywidgets as widgets
from joblib import Parallel, delayed
from google.colab import files
import joblib
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


def dataset_balance(X_temp, y_temp):
  smote = SMOTE()
  X_temp, y_temp= smote.fit_resample(X_temp, y_temp)
  return pd.concat([pd.DataFrame(X_temp), pd.DataFrame(y_temp)], axis=1)


def Xy_balance(X_temp, y_temp):
  smote = SMOTE()
  return smote.fit_resample(X_temp, y_temp)


def save_model(model,model_name):
 joblib.dump(model, model_name)
  

def load_model(model_name):
  return joblib.load(model_name)


def Search_Null(dataset):
  dic={}
  for col in list(dataset.columns):
    rows = []
    flage = 0
    for row in range(dataset.shape[0]):
      if str(dataset[col][row]) == "nan":
        rows.append(row)
        flage = 1
    if flage ==1:
      dic[col] = rows
  return dic



def divide_dataset(dataset,percentage,random_state = 0):
  rows,cols = dataset.shape
  if random_state == 0:
    return dataset.iloc[:int(rows*(percentage)),:]
  else:
    import random
    l_r = random.sample(range(0, rows), int(rows*(percentage)))
    return dataset.iloc[l_r,:]



def StandardScaleData(data):
  scaler = StandardScaler()
  scaler.fit(data)
  return  scaler.transform(data)



def MinMaxScaleData(data):
  scaler = MinMaxScaler()
  scaler.fit(data)
  return scaler.transform(data)


def tune_report_csv(model,dic,feat_selection_method,number_of_feat):

  tunning_type = []
  accuracy = []
  scaled_accuracy = []
  model_name = []
  feature_selection_method = []
  number_of_features = []

  li = ["Smote : NO    ; Scaling : Train(Standard)",
      "Smote : NO    ; Scaling : Train(MinMax)",
      "Smote : Train ; Scaling : Train(Standard)",
      "Smote : Train ; Scaling : Train(MinMax)",
      "Smote : All   ; Scaling : Train(Standard)",
      "Smote : All   ; Scaling : Train(MinMax)"]

  for i in list(dic.keys()):
    X_train, X_test, y_train, y_test = dic[i]
    
    if i in li and str(model[:3]) == "xgb":
      model.fit(X_train,y_train.values)
      y_pred = model.predict(X_test.values)
    else:
      model.fit(X_train,y_train)
      y_pred = model.predict(X_test)

    accu =accuracy_score(y_test, y_pred)
    scaled_accu =  float("{:.2f}".format(accu))
    print(i)
    print("Accurecy: ",scaled_accu)
    tunning_type.append(i)
    accuracy.append(accu)
    scaled_accuracy.append(scaled_accu)
    model_name.append(model)
    feature_selection_method.append(feat_selection_method)
    number_of_features.append(number_of_feat)
    
    print("------------------------------------------------------------------------------")
    print("------------------------------------------------------------------------------")

  csv_dic={
      "model_name"  : model_name,
      "tunning_type" : tunning_type,
      "accurecy"    : accuracy, 
      "scaled_accuracy" :scaled_accuracy,
      "feature_selection_method" : feature_selection_method,
      "number_of_features" : number_of_features
  }


  df = pd.DataFrame.from_dict(csv_dic)
  df.to_csv("/content/drive/MyDrive/Iqram Sir/Tinnitus/Features_CSV/"+feat_selection_method+"_"+str(number_of_feat)+"_"+str(model)+".csv",index = False)
  return "Tunning Data Report is Secured for this phase!!!"


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#Starting

In [69]:
dataset = pd.read_csv("/content/drive/MyDrive/Iqram Sir/Tinnitus/Tinnitus_106 feature_ALR_EEG_2023.02.08_OnlyGap_No_n0p1p3.csv")
print(dataset.shape)
target = "Class"

#Encoding
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
dataset[target]=encoder.fit_transform(dataset[target])

#dataset = divide_dataset(dataset,0.04611,1)
#print(dataset.shape)

(165, 77)


###Spliting into X and y

In [70]:
X = dataset.iloc[:,3:]
y = dataset[target]
col=list(dataset.iloc[0:0,3:])
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165 entries, 0 to 164
Data columns (total 74 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   n1_gap_peak_ms_20ms           165 non-null    int64  
 1   p2_gap_peak_ms_20ms           165 non-null    int64  
 2   n2_gap_peak_ms_20ms           165 non-null    int64  
 3   n1p2_interval_gap_20ms        165 non-null    int64  
 4   n1n2_interval_gap_20ms        165 non-null    int64  
 5   p2n2_interval_gap_20ms        165 non-null    int64  
 6   n1_gap_peak_amp_20ms          165 non-null    float64
 7   p2_gap_peak_amp_20ms          165 non-null    float64
 8   n2_gap_peak_amp_20ms          165 non-null    float64
 9   n1p2_gap_diff_20ms            165 non-null    float64
 10  p2n2_gap_diff_20ms            165 non-null    float64
 11  n1p2_gap_nogap_diff_20ms      165 non-null    float64
 12  p2n2_gap_nogap_diff_20ms      165 non-null    float64
 13  n1p2_

#Feature Selection

In [ ]:
number_of_feat = 40

###ANOVA with f classifciation

In [ ]:
from sklearn.datasets import make_classification
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
import pandas as pd



fs = SelectKBest(score_func=f_classif, k=5)
fit = fs.fit(X,y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)

featureScores = pd.concat([dfcolumns,dfscores],axis=1)

featureScores.columns = ['Best_columns','Score_ANOVA'] 

lyst = featureScores.nlargest(number_of_feat,'Score_ANOVA')

#lyst.to_csv('Filter_Method_ANOVA_with_f_classif.csv')

list_of_feat = list(lyst["Best_columns"])
selection_method = "ANOVA"

###Embedded Method

In [ ]:
from sklearn.linear_model import LassoCV
reg = LassoCV()
reg.fit(X, y)
print("Best alpha using built-in LassoCV: %f" % reg.alpha_)
print("Best score using built-in LassoCV: %f" %reg.score(X,y))
coef = pd.Series(reg.coef_, index = X.columns)

print("Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")

imp_coef = coef.sort_values()

list_of_feat=[]


for i in range(coef.shape[0]):
  if coef[i]!=0:
    list_of_feat.append(dataset.iloc[:0,i+3].name)
    
df = pd.DataFrame(list_of_feat, columns=['Best_Features'])

#df.to_csv("Embedded_Method.csv")

list_of_feat = list(df["Best_Features"])
if number_of_feat < len(list_of_feat):
  list_of_feat = list_of_feat[:number_of_feat]
selection_method = "Embedded"

Best alpha using built-in LassoCV: 1664496.378245
Best score using built-in LassoCV: 0.000418
Lasso picked 4 variables and eliminated the other 45 variables


###Pearson's with f regression

In [ ]:
from sklearn.datasets import make_regression
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
import pandas as pd


fs = SelectKBest(score_func=f_regression, k=5)
fit = fs.fit(X,y)

dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)
featureScores = pd.concat([dfcolumns,dfscores],axis=1)

featureScores.columns = ['Best_columns','Score_pearsons'] 


lyst = featureScores.nlargest(number_of_feat,'Score_pearsons')

#lyst.to_csv('Filter_Method_Pearson’s_with_f_regression.csv')

list_of_feat = list(lyst["Best_columns"])
selection_method = "Pearson"

###Sequential Feature Selection

In [ ]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
sfs = SequentialFeatureSelector(knn, n_features_to_select=number_of_feat)
sfs.fit(X, y)
list_of_feat=[]
list_of_feat=list(sfs.get_feature_names_out(X.columns))

df = pd.DataFrame(list_of_feat, columns=['Best_Features'])

#df.to_csv("Filter_Method_Sequential_feat_Selection_KNN.csv")

list_of_feat = list(df["Best_Features"])
if number_of_feat < len(list_of_feat):
  list_of_feat = list_of_feat[:number_of_feat]

selection_method = "Sequential"

#Feature list

In [ ]:
X=dataset[list_of_feat]
y=dataset[target]
print(X.shape)

In [71]:
list_of_feat2=[   
'p2_gap_peak_amp_50ms',
'p2n2_gap_diff_20ms',
'p2n2_gap_diff_50ms',
'p2n2_interval_gap_50ms',
'n2_gap_peak_amp_20ms',
'gap_auc_n1p2n2_by_total_20ms',
'n1_gap_peak_ms_20ms',
'gap_auc_n1p2n2_by_total_50ms',
'n1p2_gap_no_gap_ratio_50ms',
'gap_auc_p2n2_UP_20ms',
'gap_auc_p2n2_UP_50ms',
'n2_gap_peak_ms_50ms',
'gap_auc_n1p2n2_20ms',
'n1p2_gap_diff_50ms',
'gap_auc_total_abs_20ms',
'n1_gap_peak_ms_50ms',
'n1p2_gap_no_gap_ratio_20ms',
'alpha_peak_auc_50ms',
'gamma_peak_freq_50ms',
'ABR_peak_auc_50ms'
    ]


list_of_feat3=[  
'p2n2_gap_diff_50ms',
'p2_gap_peak_amp_50ms',
'gap_auc_n1p2n2_by_total_20ms',
'gap_auc_n1p2n2_by_total_50ms',
'p2n2_gap_diff_20ms',
'n1_gap_peak_ms_20ms',
'n2_gap_peak_amp_20ms',
'gap_auc_n1p2n2_50ms',
'gap_auc_n1p2_UP_50ms',
'gap_auc_n1p2_LO_50ms',
'gap_auc_n1p2n2_20ms',
'n1_gap_peak_ms_50ms',
'gap_auc_p2n2_UP_50ms',
'n1p2_gap_no_gap_ratio_20ms',
'p2_gap_peak_amp_20ms',
'alpha_peak_auc_50ms',
'gamma_peak_freq_20ms'
    ]

selection_method="given_list_2"

number_of_feat = len(list_of_feat2)


X=dataset[list_of_feat2]
y=dataset[target]

In [72]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165 entries, 0 to 164
Data columns (total 20 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   p2_gap_peak_amp_50ms          165 non-null    float64
 1   p2n2_gap_diff_20ms            165 non-null    float64
 2   p2n2_gap_diff_50ms            165 non-null    float64
 3   p2n2_interval_gap_50ms        165 non-null    int64  
 4   n2_gap_peak_amp_20ms          165 non-null    float64
 5   gap_auc_n1p2n2_by_total_20ms  165 non-null    float64
 6   n1_gap_peak_ms_20ms           165 non-null    int64  
 7   gap_auc_n1p2n2_by_total_50ms  165 non-null    float64
 8   n1p2_gap_no_gap_ratio_50ms    165 non-null    float64
 9   gap_auc_p2n2_UP_20ms          165 non-null    float64
 10  gap_auc_p2n2_UP_50ms          165 non-null    float64
 11  n2_gap_peak_ms_50ms           165 non-null    int64  
 12  gap_auc_n1p2n2_20ms           165 non-null    float64
 13  n1p2_

#Tunning for Scalling and Data Balancing

##Smote : NO
##Scaling : NO

In [73]:
def tune_1():
  X_new = X
  y_new = y
  return train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)

##Smote : NO
##Scaling : All Data (Standard)

In [74]:
def tune_2():
  X_new = StandardScaleData(X)
  y_new = y
  return train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)

##Smote : NO
##Scaling : All Data (MinMax)

In [75]:
def tune_3():
  X_new = MinMaxScaleData(X)
  y_new = y
  return train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)

##Smote : NO
##Scaling : Train (Stadard)

In [76]:
def tune_4():
  X_new = X
  y_new = y
  X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)
  X_train = StandardScaleData(X_train)
  return X_train, X_test, y_train, y_test

##Smote : NO
##Scaling : Train (MinMax)

In [77]:

def tune_5():
  X_new = X
  y_new = y
  X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)
  X_train = MinMaxScaleData(X_train)
  return X_train, X_test, y_train, y_test

##Smote : ALL
##Scaling : NO

In [78]:

def tune_6():
  X_new,y_new = Xy_balance(X,y)
  return train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)


##Smote : Train
##Scaling : NO

In [79]:

def tune_7():
  X_new = X
  y_new = y
  X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)
  X_train,y_train = Xy_balance(X_train,y_train)
  return X_train, X_test, y_train, y_test


##Smote : Train
##Scaling : Train (Standard)

In [80]:
def tune_8():
  X_new = X
  y_new = y
  X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)
  X_train,y_train = Xy_balance(X_train,y_train)
  X_train = StandardScaleData(X_train)
  return X_train, X_test, y_train, y_test

##Smote : Train
##Scaling : Train (MixMax)

In [81]:
def tune_9():
  X_new = X
  y_new = y
  X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)
  X_train,y_train = Xy_balance(X_train,y_train)
  X_train = MinMaxScaleData(X_train)
  return X_train, X_test, y_train, y_test

##Smote : All
##Scaling : Train (Standard)

In [82]:
def tune_10():
  X_new,y_new = Xy_balance(X,y)
  X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)
  X_train = StandardScaleData(X_train)
  return X_train, X_test, y_train, y_test 

##Smote : All
##Scaling : Train (MixMax)

In [83]:
def tune_11():
  X_new,y_new = Xy_balance(X,y)
  X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)
  X_train = MinMaxScaleData(X_train)
  return X_train, X_test, y_train, y_test 

##Smote : All
##Scaling : All (Standard)

In [84]:
def tune_12():
  X_new,y_new = Xy_balance(X,y)
  X_new = StandardScaleData(X_new)
  return train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)

##Smote : All
##Scaling : ALL (MinMax)

In [85]:
def tune_13():
  X_new,y_new = Xy_balance(X,y)
  X_new = MinMaxScaleData(X_new)
  return train_test_split(X_new, y_new, test_size = 0.2, random_state = 0)

##Tunn Dictionary

In [86]:
tune_dic={
  "Smote : NO    ; Scaling : NO"                  : tune_1(),
  "Smote : NO    ; Scaling : All Data(Standard)"  : tune_2(),
  "Smote : NO    ; Scaling : All Data(MinMax)"    : tune_3(),
  #"Smote : NO    ; Scaling : Train(Standard)"     : tune_4(),
  #"Smote : NO    ; Scaling : Train(MinMax)"       : tune_5(),
  "Smote : ALL   ; Scaling : NO"                  : tune_6(),
  "Smote : Train ; Scaling : NO"                  : tune_7(),
  #"Smote : Train ; Scaling : Train(Standard)"     : tune_8(),
  #"Smote : Train ; Scaling : Train(MinMax)"       : tune_9(),
  #"Smote : All   ; Scaling : Train(Standard)"     : tune_10(),
  #"Smote : All   ; Scaling : Train(MinMax)"       : tune_11(),
  "Smote : All   ; Scaling : All(Standard)"       : tune_12(),
  "Smote : All   ; Scaling : All(MinMax)"        : tune_13()
}



---



# **Training**

##ADABOOST

In [87]:
%%time
from sklearn.ensemble import AdaBoostClassifier
ada_defult = AdaBoostClassifier(random_state=0)
tune_report_csv(ada_defult,tune_dic,selection_method,number_of_feat)


Smote : NO    ; Scaling : NO
Accurecy:  0.67
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(Standard)
Accurecy:  0.67
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(MinMax)
Accurecy:  0.67
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : ALL   ; Scaling : NO
Accurecy:  0.66
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : Train ; Scaling : NO
Accurecy:  0.7
------------------------------------------------------------------------------
-----------------------------------

'Tunning Data Report is Secured for this phase!!!'



---



---



##Graddient Boosting

In [88]:
%%time
from sklearn.ensemble import GradientBoostingClassifier
gradBoost_default = GradientBoostingClassifier(random_state=0)

tune_report_csv(gradBoost_default,tune_dic,selection_method,number_of_feat)

Smote : NO    ; Scaling : NO
Accurecy:  0.7
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(Standard)
Accurecy:  0.7
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(MinMax)
Accurecy:  0.7
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : ALL   ; Scaling : NO
Accurecy:  0.69
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : Train ; Scaling : NO
Accurecy:  0.67
------------------------------------------------------------------------------
-------------------------------------

'Tunning Data Report is Secured for this phase!!!'

###Histogram-Based Gradient Boosting

In [89]:
%%time
from sklearn.ensemble import HistGradientBoostingClassifier

hisgradBoost_default = HistGradientBoostingClassifier(random_state=0)

tune_report_csv(hisgradBoost_default,tune_dic,selection_method,number_of_feat)

Smote : NO    ; Scaling : NO
Accurecy:  0.73
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(Standard)
Accurecy:  0.73
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(MinMax)
Accurecy:  0.73
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : ALL   ; Scaling : NO
Accurecy:  0.8
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : Train ; Scaling : NO
Accurecy:  0.7
------------------------------------------------------------------------------
------------------------------------

'Tunning Data Report is Secured for this phase!!!'



---



---



##Random Forest

In [90]:
%%time
RanFor={}

from sklearn.ensemble import RandomForestClassifier
rf_default = RandomForestClassifier(random_state=0,n_jobs=-1)

tune_report_csv(rf_default,tune_dic,selection_method,number_of_feat)

Smote : NO    ; Scaling : NO
Accurecy:  0.76
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(Standard)
Accurecy:  0.76
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(MinMax)
Accurecy:  0.79
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : ALL   ; Scaling : NO
Accurecy:  0.8
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : Train ; Scaling : NO
Accurecy:  0.76
------------------------------------------------------------------------------
-----------------------------------

'Tunning Data Report is Secured for this phase!!!'



---



---



##XGB

In [91]:
%%time
import xgboost as xgb
xgb_deafult = xgb.XGBClassifier(random_state=0,n_jobs=-1)

tune_report_csv(xgb_deafult,tune_dic,selection_method,number_of_feat)

Smote : NO    ; Scaling : NO
Accurecy:  0.76
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(Standard)
Accurecy:  0.76
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(MinMax)
Accurecy:  0.76
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : ALL   ; Scaling : NO
Accurecy:  0.69
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : Train ; Scaling : NO
Accurecy:  0.7
------------------------------------------------------------------------------
-----------------------------------

'Tunning Data Report is Secured for this phase!!!'

##KNN

In [92]:
%%time
from sklearn.neighbors import KNeighborsClassifier
knn_default = KNeighborsClassifier(n_jobs = -1)

tune_report_csv(knn_default,tune_dic,selection_method,number_of_feat)

Smote : NO    ; Scaling : NO
Accurecy:  0.7
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(Standard)
Accurecy:  0.7
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(MinMax)
Accurecy:  0.67
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : ALL   ; Scaling : NO
Accurecy:  0.57
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : Train ; Scaling : NO
Accurecy:  0.7
------------------------------------------------------------------------------
-------------------------------------

'Tunning Data Report is Secured for this phase!!!'

##NB

In [93]:
%%time
from sklearn.naive_bayes import GaussianNB

nb_deafult = GaussianNB()

tune_report_csv(nb_deafult,tune_dic,selection_method,number_of_feat)

Smote : NO    ; Scaling : NO
Accurecy:  0.73
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(Standard)
Accurecy:  0.73
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(MinMax)
Accurecy:  0.73
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : ALL   ; Scaling : NO
Accurecy:  0.66
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : Train ; Scaling : NO
Accurecy:  0.73
------------------------------------------------------------------------------
----------------------------------

'Tunning Data Report is Secured for this phase!!!'

##Tree algorithms

###DecisionTreeRegressor

In [94]:
%%time
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier()
tune_report_csv(dtc,tune_dic,selection_method,number_of_feat)

Smote : NO    ; Scaling : NO
Accurecy:  0.76
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(Standard)
Accurecy:  0.79
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : NO    ; Scaling : All Data(MinMax)
Accurecy:  0.76
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : ALL   ; Scaling : NO
Accurecy:  0.69
------------------------------------------------------------------------------
------------------------------------------------------------------------------
Smote : Train ; Scaling : NO
Accurecy:  0.7
------------------------------------------------------------------------------
-----------------------------------

'Tunning Data Report is Secured for this phase!!!'

In [95]:
!pip install catboost

from catboost import CatBoostClassifier
#cat = CatBoostClassifier(task_type="GPU")
cat=CatBoostClassifier()

tune_report_csv(cat,tune_dic,selection_method,number_of_feat)

Streaming output truncated to the last 5000 lines.
24:	learn: 0.6444270	total: 48.6ms	remaining: 1.9s
25:	learn: 0.6429376	total: 50.5ms	remaining: 1.89s
26:	learn: 0.6410398	total: 52.3ms	remaining: 1.89s
27:	learn: 0.6391613	total: 54.2ms	remaining: 1.88s
28:	learn: 0.6382177	total: 56ms	remaining: 1.87s
29:	learn: 0.6361102	total: 57.8ms	remaining: 1.87s
30:	learn: 0.6344420	total: 59.6ms	remaining: 1.86s
31:	learn: 0.6331252	total: 61.5ms	remaining: 1.86s
32:	learn: 0.6313058	total: 63.3ms	remaining: 1.86s
33:	learn: 0.6300249	total: 65.2ms	remaining: 1.85s
34:	learn: 0.6279367	total: 67.1ms	remaining: 1.85s
35:	learn: 0.6264088	total: 68.9ms	remaining: 1.84s
36:	learn: 0.6256436	total: 70.8ms	remaining: 1.84s
37:	learn: 0.6243024	total: 72.6ms	remaining: 1.84s
38:	learn: 0.6225281	total: 74.4ms	remaining: 1.83s
39:	learn: 0.6202756	total: 76.1ms	remaining: 1.82s
40:	learn: 0.6188989	total: 77.8ms	remaining: 1.82s
41:	learn: 0.6173460	total: 79.7ms	remaining: 1.82s
42:	learn: 0.615

'Tunning Data Report is Secured for this phase!!!'

#Result Analysis

In [96]:
import pandas as pd
import os
import glob


csv_files = glob.glob(os.path.join("/content/drive/MyDrive/Iqram Sir/Tinnitus/Features_CSV", "*.csv"))

final_dataset = pd.DataFrame()

for f in csv_files:
            df_temp = pd.read_csv(f)
            final_dataset = final_dataset.append(df_temp, ignore_index=True)


##Sorting according to accuracy

In [97]:
final_dataset = final_dataset.sort_values("accurecy",ascending=False,kind="mergesort")
final_dataset = final_dataset.reset_index(drop = True)
final_dataset.head(10)

##Saving

In [101]:
final_dataset.to_csv("/content/drive/MyDrive/Iqram Sir/Tinnitus/FSR.csv",index=False)

In [99]:
final_dataset.head(10)

,model_name,tunning_type,accurecy,scaled_accuracy,feature_selection_method,number_of_features
0,<catboost.core.CatBoostClassifier object at 0x...,Smote : ALL ; Scaling : NO,0.828571,0.83,given_list_2,20
1,HistGradientBoostingClassifier(random_state=0),Smote : ALL ; Scaling : NO,0.800000,0.80,given_list_2,20
2,HistGradientBoostingClassifier(random_state=0),Smote : All ; Scaling : All(Standard),0.800000,0.80,given_list_2,20
3,"RandomForestClassifier(n_jobs=-1, random_state=0)",Smote : ALL ; Scaling : NO,0.800000,0.80,given_list_2,20
4,"RandomForestClassifier(n_jobs=-1, random_state=0)",Smote : NO ; Scaling : All Data(MinMax),0.787879,0.79,given_list_2,20
5,DecisionTreeClassifier(),Smote : NO ; Scaling : All Data(Standard),0.787879,0.79,given_list_2,20
6,AdaBoostClassifier(random_state=0),Smote : All ; Scaling : All(MinMax),0.771429,0.77,given_list_2,20
7,XGBClassifier(n_jobs=-1),Smote : All ; Scaling : All(MinMax),0.771429,0.77,given_list_2,20
8,KNeighborsClassifier(n_jobs=-1),Smote : All ; Scaling : All(MinMax),0.771429,0.77,given_list_2,20
9,<catboost.core.CatBoostClassifier object at 0x...,Smote : All ; Scaling : All(Standard),0.771429,0.77,given_list_2,20
